In [16]:
import pandas as pd
import numpy as np

- **0**: 1-29 days past due - **dias de atraso** 
- **1**: 30-59 days past due - **dias de atraso** 
- **2**: 60-89 days overdue - **dias de atraso**
- **3**: 90-119 days overdue - **dias de atraso**
- **4**: 120-149 days overdue - **dias de atraso**
- **5**: more than 150 days - **mais de 150 dias**
- **C**: paid off that month - **pago naquele mês**
- **X**: No loan for the month - **Nenhum empréstimo no mês**


## 01) Marcar *default* no mês

In [41]:
dados=pd.read_csv('pagamentos_largo.csv')
proposta=pd.read_csv('application_record.csv')
print(proposta.shape)
print(dados.shape)
proposta.head()

(438557, 18)
(20937, 26)


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


## 02) 'bons' e 'maus' ao longo de todos os 24 meses de desempenho

In [35]:
dados.loc[dados.isin(['2','3','4','5']).any(axis=1),'default']='mau'
dados.loc[~dados.isin(['2','3','4','5']).any(axis=1),'default']='bom'

In [24]:
dados.default.value_counts()

bom    20506
mau      431
Name: default, dtype: int64

In [22]:
dados[dados['default']==False]

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9,default
0,5001718,0,0,0,0,0,0,0,0,NaN,...,0,NaN,0,0,0,0,NaN,0,NaN,False
1,5001719,0,0,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,False
2,5001720,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,False
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,NaN,NaN,False
4,5001726,0,0,C,C,C,C,C,C,C,...,C,C,0,0,0,C,C,C,C,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20932,5150475,C,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,False
20933,5150476,0,0,0,0,0,0,0,0,0,...,NaN,NaN,0,0,0,0,0,0,0,False
20934,5150480,0,0,C,C,C,C,C,C,C,...,C,C,0,0,0,C,C,C,C,False
20935,5150482,0,0,0,0,C,C,C,C,C,...,NaN,NaN,0,0,0,0,0,0,0,False


## 03/04) Marcando proponentes expostos ao risco/Consolidando as informações

In [39]:
df=pd.merge(proposta,dados,on='ID')
df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9,default
0,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,...,C,C,NaN,0,NaN,NaN,NaN,0,NaN,bom
1,5008810,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,C,C,0,0,NaN,0,NaN,0,NaN,bom
2,5008811,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,C,C,0,0,NaN,0,NaN,0,NaN,bom
3,5112956,M,Y,Y,0,270000.0,Working,Higher education,Married,House / apartment,...,0,0,0,0,0,0,0,0,NaN,bom
4,5008825,F,Y,N,0,130500.0,Working,Incomplete higher,Married,House / apartment,...,0,0,0,0,1,0,0,0,0,bom


## 05) Verificando valores default

In [38]:
df['default'].value_counts()

bom    16260
mau      390
Name: default, dtype: int64